In [1]:
# !pip install trectools
import pandas as pd
import trectools as trec

In [2]:
results_df = pd.read_csv("./MQ2008-agg/result_agg_functions.csv", sep="\t")
results_df.head(5)

,qid,docid,relevance,ranking_1,ranking_2,ranking_3,ranking_4,ranking_5,ranking_6,ranking_7,...,ranking_19_rank,ranking_20_rank,ranking_21_rank,ranking_22_rank,ranking_23_rank,ranking_24_rank,ranking_25_rank,rrf,squared_rrf,mrrf
0,10002,GX008-86-4444840,0,1,30,48,133,0,265,0,...,0,0,0,1,1,0,0,0.195416,9.472222,2.522652
1,10002,GX037-06-11625428,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0.049180,3.000000,0.639022
2,10002,GX044-30-4142998,0,0,0,0,0,0,8,0,...,0,0,0,4,0,0,0,0.144178,3.708333,1.831318
3,10002,GX228-42-3888699,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0.049180,3.000000,0.639022
4,10002,GX229-14-12863205,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,2,0.032258,0.500000,0.412383


In [3]:
qrel_df = results_df[["qid", "docid", "relevance"]].copy()
qrel_df['rank'] = qrel_df.groupby('qid')['relevance'].rank(method='dense', ascending=False).astype(int)
qrel_df['q0']  = 'Q0'
qrel_df['rel'] = qrel_df['relevance']
qrel_df['query'] = qrel_df['qid']
qrel_df = qrel_df[["qid","query", "q0", "docid", "rel"]]
print(qrel_df.head(5))
trec_qrel = trec.TrecQrel()
trec_qrel.qrels_data = qrel_df.copy()

     qid  query  q0              docid  rel
0  10002  10002  Q0   GX008-86-4444840    0
1  10002  10002  Q0  GX037-06-11625428    0
2  10002  10002  Q0   GX044-30-4142998    0
3  10002  10002  Q0   GX228-42-3888699    0
4  10002  10002  Q0  GX229-14-12863205    0


In [4]:
# Evaluate RRF
run_df_rrf = results_df[["qid", "docid", "rrf"]].copy()
run_df_rrf['rank'] = run_df_rrf.groupby('qid')['rrf'].rank(method='dense', ascending=False).astype(int)
run_df_rrf['q0']  = 'Q0'
run_df_rrf['score'] = run_df_rrf['rrf']
run_df_rrf['system'] = 'rrf'
run_df_rrf['query'] = run_df_rrf['qid']
run_df_rrf = run_df_rrf[["qid","query","q0", "docid","rank" ,"score", "system"]]
# print(run_df_rrf.head(5))

# RRF Evaluation
run_rrf = trec.TrecRun()
run_rrf.run_data = run_df_rrf.copy()
rrf_trec_eval = trec.TrecEval(run_rrf, trec_qrel)
print(f"rrd:{rrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"ndcg@{i}:{rrf_trec_eval.get_ndcg(depth=i, trec_eval=False)}")
    print(f"precision@{i}:{rrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"map@{i}:{rrf_trec_eval.get_map(depth=i, trec_eval=False)}")
    


rrd:2932
ndcg@1:0.13903061224489796
precision@1:0.16454081632653061
map@1:0.05409057112031064
ndcg@2:0.17083915819161816
precision@2:0.19260204081632654
map@2:0.09330295170037976
ndcg@3:0.20102492437063615
precision@3:0.20493197278911562
map@3:0.1258940875651836
ndcg@4:0.2311989719984189
precision@4:0.21205357142857142
map@4:0.15544909486087932
ndcg@5:0.2560544289674326
precision@5:0.21173469387755103
map@5:0.17932334116455
ndcg@6:0.2809029882840863
precision@6:0.21045918367346939
map@6:0.19997965921647268
ndcg@7:0.305329402200886
precision@7:0.21191690962099122
map@7:0.2222365775991628
ndcg@8:0.3245239985823
precision@8:0.2074298469387755
map@8:0.23879718863940372
ndcg@9:0.3296799519529564
precision@9:0.19387755102040816
map@9:0.24324573303864874
ndcg@10:0.3346559257223674
precision@10:0.18252551020408167
map@10:0.24737064167917353


In [5]:
# Evaluate SRRF
run_df_srrf = results_df[["qid", "docid", "squared_rrf"]].copy()
run_df_srrf['rank'] = run_df_srrf.groupby('qid')['squared_rrf'].rank(method='dense', ascending=False).astype(int)
run_df_srrf['q0']  = 'Q0'
run_df_srrf['score'] = run_df_srrf['squared_rrf']
run_df_srrf['system'] = 'srrf'
run_df_srrf['query'] = run_df_srrf['qid']
run_df_srrf = run_df_srrf[["qid","query","q0", "docid","rank" ,"score", "system"]]
# print(run_df_srrf.head(5))

# SRRF Evaluation
run_srrf = trec.TrecRun()
run_srrf.run_data = run_df_srrf.copy()
srrf_trec_eval = trec.TrecEval(run_srrf, trec_qrel)
print(f"rrd:{srrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    # print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i)}")
    print(f"ndcg@{i}:{srrf_trec_eval.get_ndcg(depth=i, trec_eval=False)}")
    print(f"precision@{i}:{srrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"map@{i}:{srrf_trec_eval.get_map(depth=i, trec_eval=False)}")
    

rrd:2932
ndcg@1:0.13903061224489796
precision@1:0.16454081632653061
map@1:0.05409057112031064
ndcg@2:0.17083915819161816
precision@2:0.19260204081632654
map@2:0.09330295170037976
ndcg@3:0.20102492437063615
precision@3:0.20493197278911562
map@3:0.1258940875651836
ndcg@4:0.2311989719984189
precision@4:0.21205357142857142
map@4:0.15544909486087932
ndcg@5:0.2560544289674326
precision@5:0.21173469387755103
map@5:0.17932334116455
ndcg@6:0.2809029882840863
precision@6:0.21045918367346939
map@6:0.19997965921647268
ndcg@7:0.305329402200886
precision@7:0.21191690962099122
map@7:0.2222365775991628
ndcg@8:0.3245239985823
precision@8:0.2074298469387755
map@8:0.23879718863940372
ndcg@9:0.3296799519529564
precision@9:0.19387755102040816
map@9:0.24324573303864874
ndcg@10:0.3346559257223674
precision@10:0.18252551020408167
map@10:0.24737064167917353


In [6]:
# Evaluate MRRF
run_df_mrrf = results_df[["qid", "docid", "mrrf"]].copy()
run_df_mrrf['rank'] = run_df_mrrf.groupby('qid')['mrrf'].rank(method='dense', ascending=False).astype(int)
run_df_mrrf['q0']  = 'Q0'
run_df_mrrf['score'] = run_df_mrrf['mrrf']
run_df_mrrf['system'] = 'mrrf'
run_df_mrrf['query'] = run_df_mrrf['qid']
run_df_mrrf = run_df_mrrf[["qid","query","q0", "docid","rank", "score", "system"]]
# print(run_df_mrrf.head(5))

# MRRF Evaluation
run_mrrf = trec.TrecRun()
run_mrrf.run_data = run_df_mrrf.copy()
mrrf_trec_eval = trec.TrecEval(run_mrrf, trec_qrel)
print(f"rrd:{mrrf_trec_eval.get_relevant_retrieved_documents()}")
for i in range(1,11):
    print(f"ndcg@{i}:{mrrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"precision@{i}:{mrrf_trec_eval.get_precision(depth=i, trec_eval=False)}")
    print(f"map@{i}:{mrrf_trec_eval.get_map(depth=i, trec_eval=False)}")
    

rrd:2932
ndcg@1:0.16454081632653061
precision@1:0.16454081632653061
map@1:0.05409057112031064
ndcg@2:0.19260204081632654
precision@2:0.19260204081632654
map@2:0.09330295170037976
ndcg@3:0.20493197278911562
precision@3:0.20493197278911562
map@3:0.1258940875651836
ndcg@4:0.21205357142857142
precision@4:0.21205357142857142
map@4:0.15544909486087932
ndcg@5:0.21173469387755103
precision@5:0.21173469387755103
map@5:0.17932334116455
ndcg@6:0.21045918367346939
precision@6:0.21045918367346939
map@6:0.19997965921647268
ndcg@7:0.21191690962099122
precision@7:0.21191690962099122
map@7:0.2222365775991628
ndcg@8:0.2074298469387755
precision@8:0.2074298469387755
map@8:0.23879718863940372
ndcg@9:0.19387755102040816
precision@9:0.19387755102040816
map@9:0.24324573303864874
ndcg@10:0.18252551020408167
precision@10:0.18252551020408167
map@10:0.24737064167917353
